#### 1. Loading Processed Data

In [11]:
import numpy as np
import os

# Path relative to notebook directory
base_path = "../data/processed"

X_train = np.load(os.path.join(base_path, "X_train_processed.npy"))
X_test = np.load(os.path.join(base_path, "X_test_processed.npy"))
y_train = np.load(os.path.join(base_path, "y_train_encoded.npy"))
y_test = np.load(os.path.join(base_path, "y_test_encoded.npy"))

print(X_train.shape, y_train.shape)

(87000, 64, 64) (87000,)


#### 2. Initial Setup

##### 2.1 Split training/validation sets

In [12]:
from sklearn.model_selection import train_test_split
# split the data into 10% validation set and 90% training set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)


In [13]:
from tensorflow.keras.utils import to_categorical # Pylance might still warn, but it's correct
# One hot encoding to handle categorical formatting
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

##### 2.2 Building CNN model

In [14]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


e:\Ashwin\importantFiles\Programming Projects\WASL-translator\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 686,365 (2.62 MB)

 Trainable params: 686,365 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

#### 3. Training Model

In [15]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_val, y_val)
)

Epoch 1/20
 289/1224 ━━━━━━━━━━━━━━━━━━━━ 1:08 73ms/step - accuracy: 0.1030 - loss: 4.8098

KeyboardInterrupt: 

### 4. Evaluation

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save("../models/asl_cnn_model.h5")

In [ ]:
import pickle

with open("../models/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)